In [13]:
import pennylane as qml 
import pennylane.numpy as pnp 
import numpy as np 
from qas import qml_ops 
from qas import qml_models 
from qas import mcts
import os 
import json 
from pprint import pprint

dev = qml.device('default.qubit', wires=3)

In [14]:
cwd = os.getcwd() 
print(cwd)

res_file = "20211005-114848.json"
with open(os.path.join(cwd, res_file)) as f:
    res_dict = json.load(f)
print(res_dict.keys())

d:\Research\QAS-Qiskit\QAS
dict_keys(['task', 'pool', 'params', 'k', 'op_list', 'search_reward_list', 'fine_tune_loss'])


In [15]:
pprint(res_dict['op_list'])

[['CRot',
  [0, 1],
  [0.7155452974765524, 0.020417804916402466, -0.713502489836734]],
 ['CRot',
  [1, 2],
  [1.5190876497379346, -0.2546445896530236, -1.5184202417063677]],
 ['CNOT', [0, 2], None],
 ['Rot', [0], [-1.5622544796698883, -0.5413672571695849, 1.5688514279418972]],
 ['Rot', [2], [-1.5706971253874478, 0.4075454388572003, 1.5707010598740383]],
 ['Rot', [1], [-1.7295778023146682, -2.677612052206618, 1.5374237066523149]],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None]]


In [16]:
@qml.qnode(dev)
def circ(params, x=None):
    qml.QubitStateVector(x, wires=[0,1,2])
    
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.Rot(params[6], params[7], params[8], wires=[0])
    qml.Rot(params[9], params[10], params[11], wires=[2])
    qml.Rot(params[12], params[13], params[14], wires=[1])
   
    return qml.state()


@qml.qnode(dev)
def circ_toffoli(x=None):
    qml.QubitStateVector(x, wires=[0,1,2])
    qml.Toffoli(wires=[0,1,2])
   
    return qml.state()

dev2 = qml.device('default.qubit', wires=7)

@qml.qnode(dev2)
def circ_swap_test(params, x=None):

    qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    #for i in range(6):
    #    qml.Hadamard(wires=i)

    #qml.Toffoli(wires=[0,1,2])

    
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.Rot(params[6], params[7], params[8], wires=[0])
    qml.Rot(params[9], params[10], params[11], wires=[2])
    qml.Rot(params[12], params[13], params[14], wires=[1])
    
    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))




@qml.qnode(dev2)
def circ_swap_test_fake(params, x=None):

    qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    #for i in range(6):
    #    qml.Hadamard(wires=i)

    qml.Toffoli(wires=[0,1,2])

    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))

@qml.qnode(dev2)
def circ_swap_test_full_hadamard(params):

    #qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    for i in range(6):
        qml.Hadamard(wires=i)
    #qml.Hadamard(wires=0)
    #qml.Hadamard(wires=1)

    #qml.Hadamard(wires=3)
    #qml.Hadamard(wires=4)
    #qml.Toffoli(wires=[0,1,2])

    
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.Rot(params[6], params[7], params[8], wires=[0])
    qml.Rot(params[9], params[10], params[11], wires=[2])
    qml.Rot(params[12], params[13], params[14], wires=[1])
    
    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))

def cost_swap_test(params):
    x_data = qml_models.TOFFOLI_DATA[0]
    fid = 0
    for i in range(len(x_data)):
        out=circ_swap_test(params, x=x_data[i])
        fid = fid + (out-1/2)*2
    return 1-fid/len(x_data)

def cost_swap_test_full_hadamard(params):
    fid = (circ_swap_test_full_hadamard(params)-1/2)*2
    return 1-fid



In [17]:
np.random.seed(0)
var = np.random.randn(15)*0.01

#print(qml.draw(circ_swap_test_full_hadamard)(var))

opt = qml.AdamOptimizer(0.5)
num_epochs = 100
for epoch in range(num_epochs):
    var = opt.step(lambda v: cost_swap_test_full_hadamard(v), var)
    loss = cost_swap_test_full_hadamard(var)
    print("Epoch: {}, Loss: {}".format(epoch+1, loss))

print(qml.draw(circ_swap_test_full_hadamard)(var))

Epoch: 1, Loss: 0.8529455215794299
Epoch: 2, Loss: 0.5748156755431164
Epoch: 3, Loss: 0.1801809916990902
Epoch: 4, Loss: 0.20629756323430226
Epoch: 5, Loss: 0.3459821196998414
Epoch: 6, Loss: 0.2785829612081512
Epoch: 7, Loss: 0.146506045279722
Epoch: 8, Loss: 0.1332246644922812
Epoch: 9, Loss: 0.164334188099317
Epoch: 10, Loss: 0.17062160322417363
Epoch: 11, Loss: 0.12760166210126145
Epoch: 12, Loss: 0.08427643219220271
Epoch: 13, Loss: 0.09550425232839754
Epoch: 14, Loss: 0.10232747848569623
Epoch: 15, Loss: 0.05888249062558959
Epoch: 16, Loss: 0.03082571318505778
Epoch: 17, Loss: 0.05495638979298656
Epoch: 18, Loss: 0.06430730354220682
Epoch: 19, Loss: 0.03726621324295798
Epoch: 20, Loss: 0.02541310728462376
Epoch: 21, Loss: 0.041707204399318076
Epoch: 22, Loss: 0.04536767494169269
Epoch: 23, Loss: 0.031324688160444314
Epoch: 24, Loss: 0.024326750378737882
Epoch: 25, Loss: 0.02568312894387903
Epoch: 26, Loss: 0.02625795035690559
Epoch: 27, Loss: 0.020928935669615623
Epoch: 28, Loss:

In [18]:
for x in qml_models.TOFFOLI_DATA[0]:
    print("X:")
    print(x)
    print(circ(var, x))
    print(circ_toffoli(x))
    print("Fidelity: {}".format((circ_swap_test(var, x)-1/2)*2))
    

X:
[1 0 0 0 0 0 0 0]
[ 1.0000e+00-5.6010e-04j  3.6676e-04+1.0943e-05j -2.5861e-03+1.5135e-03j -9.6535e-07+5.2627e-07j  6.1469e-04+4.9335e-05j  2.2489e-07+2.4947e-08j -1.6648e-06+8.0185e-07j -6.1951e-10+2.7553e-10j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Fidelity: 0.9999905062897183
X:
[0 1 0 0 0 0 0 0]
[-3.6673e-04+1.2027e-05j  9.9999e-01-2.3972e-03j  9.3052e-07-5.8564e-07j -2.5833e-03+1.5183e-03j -2.2601e-07-1.0826e-08j  6.1478e-04+4.8206e-05j  6.0106e-10-3.1375e-10j -1.6633e-06+8.0491e-07j]
[0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Fidelity: 0.9999905062897185
X:
[0 0 1 0 0 0 0 0]
[2.5899e-03+1.5070e-03j 5.1165e-06-2.2322e-06j 9.9999e-01-1.9526e-03j 1.0982e-03-1.5048e-03j 1.5171e-06+1.0550e-06j 3.2559e-09-1.1179e-09j 6.1476e-04+4.8479e-05j 7.4981e-07-8.7039e-07j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Fidelity: 0.9999871703984791
X:
[0 0 0 1 0 0 0 0]
[-6.0060e-07-5.5498e-06j  2.5935e-03+1.5009e-03j -1.1077e-03-1.4979e-03j  9.9998e